In [ ]:
import numpy as np
import psrqpy

from pywwt.jupyter import WWTJupyterWidget

from os import path

from astropy import units as u
from astroquery.simbad import Simbad
from astroquery.skyview import SkyView
from astropy.coordinates import Angle, SkyCoord
from astropy.constants import c
from astropy.table import Table

# Start the widget
You might want to right-click on the panel to the left of the image and select 'Create New View from Output'

In [ ]:
wwt = WWTJupyterWidget()
wwt

# Some organising
### Pointing parameters
Change as required.
Make sure you set the position to the right coordinates.

In [ ]:
# No need to have time running
wwt.pause_time()
# Turn on the galactic grid
wwt.galactic_grid = True
# Center position of the incoherent beam
ib_centre = SkyCoord(ra="0h0m0.0s", dec="+0d0m0.0s")
wwt.center_on_coordinates(ib_centre)

In [ ]:
cfreq = 1284000000.0 * u.Hz # centre frequency in Hz
antenna_d = 13.96 * u.m# antenna diameter in m

ib_rad = 1.2 * c / (antenna_d * cfreq)
ib_deg = (ib_rad * u.rad).to(u.deg)

In [ ]:
# That currently does not mean much due to projection
ib = wwt.add_circle(ib_centre, radius = ib_deg / 2, opacity=.4, fill_color="#C4D600")

### Source parameters
Change as required

In [ ]:
cand_dm = 5678
mw_dm = 1243
dm_excess = cand_dm - mw_dm
cand_z = dm_excess / 855
upper_cand_z = 1.5 * cand_z
lower_cand_z = 0.5 * cand_z

# Catalogues

### Pulsar catalogue

In [ ]:
psr_cat = psrqpy.QueryATNF(params=['F0', 'DM', 'RAJ', 'DecJ'], circular_boundary=[ib_centre.ra.to_string(unit=u.hourangle, sep=":"), ib_centre.dec.to_string(sep=":"), 20.0])

In [ ]:
found_pulsars = psr_cat.table
ra = found_pulsars["RAJ"]
dec = found_pulsars["DECJ"]

found_pulsars["RA"] = Angle(ra, u.hour)
found_pulsars["DEC"] = Angle(dec, u.deg)

In [ ]:
psr_layer = wwt.layers.add_table_layer(found_pulsars)
psr_layer.size_scale = 100
psr_layer.color = 'red'

In [ ]:
wwt.center_on_coordinates(ib_centre)

### Magnetars

In [ ]:
magnetars = Table.read("magnetars.csv")
magnetars[:5]

In [ ]:
ra = magnetars["RA"]
dec = magnetars["Dec"]

magnetars["RA"] = Angle(ra, u.hour)
magnetars["Dec"] = Angle(dec, u.deg)

In [ ]:
mag_layer = wwt.layers.add_table_layer(magnetars)
mag_layer.color = 'white'
mag_layer.size_scale = 200

### Simbad

In [ ]:
red_simbad = Simbad()
red_simbad.get_votable_fields()

In [ ]:
red_simbad.add_votable_fields("distance_result", "z_value", "otype")
red_simbad.get_votable_fields()

In [ ]:
simbad_results = red_simbad.query_region(ib_centre, radius=2*u.degree)

In [ ]:
print(simbad_results)

In [ ]:
simbad_results["RA"] = Angle(simbad_results["RA"], unit=u.hourangle)
simbad_results["DEC"] = Angle(simbad_results["DEC"], unit=u.deg)

In [ ]:
simbad_layer = wwt.layers.add_table_layer(simbad_results)

In [ ]:
simbad_layer.size_scale = 75
simbad_layer.color = "blue"
simbad_layer.opacity = 0.5

In [ ]:
# Select sources within the redshift limits
simbad_rlim = simbad_results[simbad_results["Z_VALUE"] >= lower_cand_z]
simbad_rlim = simbad_results[simbad_results["Z_VALUE"] <= upper_cand_z]
print(len(simbad_rlim))

In [ ]:
simbad_rlim_layer = wwt.layers.add_table_layer(simbad_rlim)

In [ ]:
simbad_rlim_layer.size_scale = 100
simbad_rlim_layer.color = "green"
simbad_rlim_layer.opacity = 1.0
simbad_rlim_layer.marker_type = "circle"
simbad_rlim_layer.marker_scale = "screen"

In [ ]:
simbad_rlim_within = simbad_rlim[simbad_rlim["DISTANCE_RESULT"] <= (ib_deg / 2).to(u.arcsec)]
len(simbad_rlim_within)

In [ ]:
within_layer = wwt.layers.add_table_layer(simbad_rlim_within)

In [ ]:
within_layer.size_scale = 150
within_layer.marker_type = "circle"
within_layer.marker_scale = "screen"

In [ ]:
# Print out source types we have
sim_rlim_by_type = simbad_rlim_within.group_by("OTYPE")
for key, group in zip(sim_rlim_by_type.groups.keys, sim_rlim_by_type.groups):
    print(key, len(group))
    print("\n")

In [ ]:
# Download an SDSS image for every position
positions = [str(ra) + "h " + str(dec) + "deg" for ra, dec in zip(np.array(simbad_rlim_within["RA"]), np.array(simbad_rlim_within["DEC"]))]
full_images = []

for idx, position in enumerate(positions):
    print(idx)
    img_list = SkyView.get_images(position=position, survey="SDSSu", pixels=100)
    if len(img_list) == 1:
        full_images.append(img_list[0])

In [ ]:
# Actually display the images
full_layers = []

for image in full_images:
    full_layers.append(wwt.layers.add_image_layer(image))

In [ ]:
wwt.center_on_coordinates(ib_centre)

In [ ]:
wwt.grid = True